In [1]:
import random
from lang import load_data_int_seq
from utils import eq_encoder, is_eq_valid
from typing import List
from models.rl.env import IntegerSequenceEnv, get_current_position, encode_with_lang, decode_with_lang
import gym, ray
from ray.rllib.agents import ppo


%load_ext autoreload
%autoreload 2


/usr/local/anaconda3/envs/deeplearning/lib/python3.9/site-packages/ray/autoscaler/_private/cli_logger.py:57: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  warnings.warn(


In [ ]:
ray.init()


In [2]:
output_lang, input_lang, train, X_test, y_test = load_data_int_seq()

In [3]:
input_lang.n_words

4123

In [4]:
def compare_sequences(output_sequence: List[int], target_sequence: List[int]) -> float:
    magnitude: float = 0.0
    for x, y in enumerate(zip(output_sequence, target_sequence)):
        magnitude += abs(x - y)
    # magnitude /= len(norm_target_seq)
    return magnitude


In [5]:
def evaluate_candidate_eq(candidate: str, int_seq: List[int]) -> float:
    if is_eq_valid(candidate) == False:
        return 99.0

    output_sequence = eq_to_seq(candidate, 9)

    if np.count_nonzero(output_sequence) < 1:
        return max_penalty_magnitude

    return compare_sequences(output_sequence, int_seq)

In [6]:
env = (IntegerSequenceEnvtrain[0][0], 9, input_lang, output_lang, evaluate_candidate_eq)

Tuple(Box(0, 14, (9,), int64), Box(0, 4123, (8,), int64))


In [7]:
for _ in range(0, 8):
    env.step(1)

In [8]:
env.step(1)

(([3, 3, 3, 3, 3, 3, 3, 3, 3], [2, 3, 4, 5, 6, 7, 8, 9]), 99.0, True)

In [12]:
trainer = ppo.PPOTrainer(env=IntegerSequenceEnv, config={
    "env_config": {},  # config to pass to env class
})

2021-07-06 17:12:50,760	INFO trainer.py:696 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


RayActorError: The actor died because of an error raised in its creation task, [36mray::RolloutWorker.__init__()[39m (pid=73821, ip=10.240.179.210)
  File "python/ray/_raylet.pyx", line 501, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 451, in ray._raylet.execute_task.function_executor
  File "/usr/local/anaconda3/envs/deeplearning/lib/python3.9/site-packages/ray/_private/function_manager.py", line 563, in actor_method_executor
    return method(__ray_actor, *args, **kwargs)
  File "/usr/local/anaconda3/envs/deeplearning/lib/python3.9/site-packages/ray/rllib/evaluation/rollout_worker.py", line 384, in __init__
    self.env = _validate_env(env_creator(env_context))
  File "/usr/local/anaconda3/envs/deeplearning/lib/python3.9/site-packages/ray/rllib/agents/trainer.py", line 1409, in <lambda>
    register_env(name, lambda config: env_object(config))
TypeError: __init__() missing 4 required positional arguments: 'output_length', 'input_lang', 'output_lang', and 'eval

In [ ]:
trainer.train()